# Simple CNN notebook

## Import Libraries

In [1]:
%matplotlib inline
import numpy as np
from utils.helper import read_data, morphology
from tensorflow import keras
FOLDER = "data/data_big_2/"
MORPHOLOGY_ACTIVATED = False

## Load data

In [2]:
data_empty = read_data(FOLDER + "empty.csv")
data_1p = read_data(FOLDER + "1p.csv")
data_2p = read_data(FOLDER + "2p.csv")
data_3p = read_data(FOLDER + "3p.csv")
print("----------------Shapes----------------")
print("empty:", data_empty.shape)
print("1p:", data_1p.shape)
print("2p:", data_2p.shape)
print("3p:", data_3p.shape)

----------------Shapes----------------
empty: (720, 5, 3, 64, 64)
1p: (720, 5, 3, 64, 64)
2p: (720, 5, 3, 64, 64)
3p: (720, 5, 3, 64, 64)


In [3]:
LENGTH_EMPTY = data_empty.shape[0]
LENGTH_1P = data_1p.shape[0]
LENGTH_2P = data_2p.shape[0]
LENGTH_3P = data_3p.shape[0]

In [4]:
data_empty_summed = np.abs(np.sum(data_empty, axis=1)) 
data_1p_summed = np.abs(np.sum(data_1p, axis=1))
data_2p_summed = np.abs(np.sum(data_2p, axis=1))
data_3p_summed = np.abs(np.sum(data_3p, axis=1))

In [5]:
data_empty_summed[:, :, 32, :] = 0
data_1p_summed[:, :, 32, :] = 0
data_2p_summed[:, :, 32, :] = 0
data_3p_summed[:, :, 32, :] = 0

In [6]:
if MORPHOLOGY_ACTIVATED == True:
    data_empty_summed =  morphology(data_empty_summed)
    data_1p_summed =  morphology(data_1p_summed)
    data_2p_summed =  morphology(data_2p_summed)
    data_3p_summed =  morphology(data_3p_summed)

In [7]:
data_0 = np.moveaxis(data_empty_summed, 1,3)
data_1 = np.moveaxis(data_1p_summed, 1,3)
data_2 = np.moveaxis(data_2p_summed, 1,3)
data_3 = np.moveaxis(data_3p_summed, 1,3)
print(data_0.shape)
print(data_1.shape)
print(data_2.shape)
print(data_3.shape)

(720, 64, 64, 3)
(720, 64, 64, 3)
(720, 64, 64, 3)
(720, 64, 64, 3)


In [8]:
data_tot = np.concatenate((data_0, data_1, data_2, data_3), axis=0)
data_tot.shape

(2880, 64, 64, 3)

### Create Labels

In [9]:
labels_empty = np.zeros(LENGTH_EMPTY)
labels_1p = np.ones(LENGTH_1P)
labels_2p = np.ones(LENGTH_2P) * 2
labels_3p = np.ones(LENGTH_3P) * 3
labels = np.concatenate((labels_empty, labels_1p, labels_2p, labels_3p), axis=0)
print("Labels shape:", labels.shape)

Labels shape: (2880,)


In [10]:
train_labels_one_hot = np.eye(4)[labels.astype(int)]
print("Labels abels one hot shape:", train_labels_one_hot.shape)

Labels abels one hot shape: (2880, 4)


In [11]:
from sklearn.model_selection import train_test_split
train_X, test_X, train_y, test_y = train_test_split(data_tot, train_labels_one_hot, test_size=0.2, stratify = train_labels_one_hot)
print(f"Shape train_X: {train_X.shape}")
print(f"Shape test_X: {test_X.shape}")
print(f"Shape train_y: {train_y.shape}")
print(f"Shape test_y: {test_y.shape}")

Shape train_X: (2304, 64, 64, 3)
Shape test_X: (576, 64, 64, 3)
Shape train_y: (2304, 4)
Shape test_y: (576, 4)


In [12]:
models = keras.models
layers = keras.layers

# model = models.Sequential(
#     [
#         layers.Conv2D(8,
#                       input_shape = (64,64,3),
#                       kernel_size = 5,
#                       padding = "same",
#                       activation = "relu"),
#         layers.MaxPooling2D(2,
#                             strides = None,
#                             padding = "valid"),
#         layers.Conv2D(8,
#                       input_shape = (32,32,8),
#                       kernel_size = 3,
#                       padding = "same",
#                       activation = "relu"),
#         layers.MaxPooling2D(2,
#                     strides = None,
#                     padding = "valid"),
#         layers.Conv2D(8,
#                       input_shape = (16,16,8),
#                       kernel_size = 3,
#                       padding = "same",
#                       activation = "relu"),
#         layers.MaxPooling2D(2,
#                     strides = None,
#                     padding = "valid"),
#         layers.Flatten(),
#         layers.Dense(4),
#         layers.Activation("softmax")
#     ]
# )
model = models.Sequential(
    [
        layers.Conv2D(2,
                      input_shape = (64,64,3),
                      kernel_size = 5,
                      padding = "same",
                      activation = "relu"),
        layers.MaxPooling2D(2,
                            strides = None,
                            padding = "valid"),
        layers.Conv2D(4,
                      input_shape = (32,32,8),
                      kernel_size = 4,
                      padding = "same",
                      activation = "relu"),
        layers.MaxPooling2D(2,
                    strides = None,
                    padding = "valid"),
        layers.Conv2D(8,
                      input_shape = (16,16,4),
                      kernel_size = 3,
                      padding = "same",
                      activation = "relu"),
        layers.MaxPooling2D(2,
                    strides = None,
                    padding = "valid"),
        layers.Flatten(),
        layers.Dense(4),
        layers.Activation("softmax")
    ]
)
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 64, 64, 2)         152       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 32, 32, 2)        0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 4)         132       
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 16, 16, 4)        0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 16, 16, 8)         296       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 8, 8, 8)          0

2022-05-26 14:12:28.994990: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [13]:
model.compile(
    loss = "categorical_crossentropy",
    optimizer = keras.optimizers.Adam(learning_rate = 0.001),
    metrics = ['accuracy']
)

model.fit(
    train_X, train_y,
    epochs  = 100,
    verbose = 1,
    validation_split = 0.1,
    shuffle= True,
    callbacks=[keras.callbacks.EarlyStopping(patience = 3),
               keras.callbacks.CSVLogger('history/historyCNN.csv')]
)
if MORPHOLOGY_ACTIVATED == True:
    model.save("models/CNN_morphology")
else:
    model.save("models/CNN")

Epoch 1/100
65/65 [==============================] - 3s 33ms/step - loss: 1.2192 - accuracy: 0.3623 - val_loss: 1.1245 - val_accuracy: 0.5238
Epoch 2/100
65/65 [==============================] - 2s 32ms/step - loss: 0.9202 - accuracy: 0.6565 - val_loss: 0.7551 - val_accuracy: 0.7013
Epoch 3/100
65/65 [==============================] - 2s 30ms/step - loss: 0.6663 - accuracy: 0.7221 - val_loss: 0.6789 - val_accuracy: 0.6797
Epoch 4/100
65/65 [==============================] - 2s 33ms/step - loss: 0.5428 - accuracy: 0.7665 - val_loss: 0.5253 - val_accuracy: 0.8052
Epoch 5/100
65/65 [==============================] - 2s 33ms/step - loss: 0.4868 - accuracy: 0.7935 - val_loss: 0.4552 - val_accuracy: 0.8442
Epoch 6/100
65/65 [==============================] - 2s 32ms/step - loss: 0.4254 - accuracy: 0.8230 - val_loss: 0.4348 - val_accuracy: 0.8615
Epoch 7/100
65/65 [==============================] - 2s 31ms/step - loss: 0.3816 - accuracy: 0.8471 - val_loss: 0.4147 - val_accuracy: 0.8312
Epoch 

INFO:tensorflow:Assets written to: models/CNN/assets


INFO:tensorflow:Assets written to: models/CNN/assets


In [14]:
loss, aacuracy = model.evaluate(test_X, test_y)
print(f"Loss: {loss}")
print(f"Accuracy: {aacuracy}")

18/18 [==============================] - 0s 10ms/step - loss: 0.3235 - accuracy: 0.8594
Loss: 0.32350394129753113
Accuracy: 0.859375
